<a href="https://colab.research.google.com/github/sarahgin/DeepLeaningProj/blob/master/InstanceClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# mount data
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


## Copy data from google drive to colab env

In [0]:
! mkdir /Drive
!ls /Drive

In [0]:
#!mkdir /Drive
import shutil
import glob
import os
from zipfile import ZipFile

for src in glob.glob('/content/gdrive/My Drive/videos_2/yt_bb_detection_train/*.zip'):
  dst = f'/Drive/{os.path.basename(src)}'
  print(src, dst)    
  if os.path.exists(src) and not os.path.exists(dst):
    print(f'copy {src} to {dst}')
    shutil.copy2(src, dst)
  
 # Create a ZipFile Object and load sample.zip in it
for z in glob.glob('/Drive/*zip'):
  with ZipFile(z, 'r') as zipObj:
    # Extract all the contents of zip file in current directory
    zipObj.extractall('/Drive')
    
!ls /Drive


/content/gdrive/My Drive/videos_2/yt_bb_detection_train/0.zip /Drive/0.zip
copy /content/gdrive/My Drive/videos_2/yt_bb_detection_train/0.zip to /Drive/0.zip
/content/gdrive/My Drive/videos_2/yt_bb_detection_train/1.zip /Drive/1.zip
copy /content/gdrive/My Drive/videos_2/yt_bb_detection_train/1.zip to /Drive/1.zip
/content/gdrive/My Drive/videos_2/yt_bb_detection_train/2.zip /Drive/2.zip
copy /content/gdrive/My Drive/videos_2/yt_bb_detection_train/2.zip to /Drive/2.zip
/content/gdrive/My Drive/videos_2/yt_bb_detection_train/3.zip /Drive/3.zip
copy /content/gdrive/My Drive/videos_2/yt_bb_detection_train/3.zip to /Drive/3.zip
/content/gdrive/My Drive/videos_2/yt_bb_detection_train/4.zip /Drive/4.zip
copy /content/gdrive/My Drive/videos_2/yt_bb_detection_train/4.zip to /Drive/4.zip
/content/gdrive/My Drive/videos_2/yt_bb_detection_train/5.zip /Drive/5.zip
copy /content/gdrive/My Drive/videos_2/yt_bb_detection_train/5.zip to /Drive/5.zip
/content/gdrive/My Drive/videos_2/yt_bb_detection_tr

## Data partition to train/test folders

In [0]:
import shutil
import os
import pathlib
import random

def list_dirs(directory):
    """Returns all directories in a given directory
    """
    return [f for f in pathlib.Path(directory).iterdir() if f.is_dir()]

def list_files(directory):
    """Returns all files in a given directory
    """
    return [
        f
        for f in pathlib.Path(directory).iterdir()
        if f.is_file() and not f.name.startswith(".")
    ]

def setup_files(class_dir, seed):
    """Returns shuffled files
    """
    # make sure its reproducible
    random.seed(seed)

    files = list_files(class_dir)

    files.sort()
    random.shuffle(files)
    return files

def ratio(input, output="output", seed=1337, ratio=(0.8, 0.1, 0.1)):
    # make up for some impression
    assert round(sum(ratio), 5) == 1
    assert len(ratio) in (2, 3)

    for class_dir in list_dirs(input):
        for instance_dir in list_dirs(class_dir):
            instancename = os.path.basename(instance_dir)
            classname = os.path.basename(class_dir)
            fulloutput = os.path.join(classname,instancename)
            #output_dir = os.path.join(output, classname ,instancename)
            split_class_dir_ratio(instance_dir, output, fulloutput, ratio, seed, None)


def split_class_dir_fixed(class_dir, output, fixed, seed, prog_bar):
    """Splits one very class folder
    """
    files = setup_files(class_dir, seed)

    if not len(files) > sum(fixed):
        raise ValueError(
            f'The number of samples in class "{class_dir.stem}" are too few. There are only {len(files)} samples available but your fixed parameter {fixed} requires at least {sum(fixed)} files. You may want to split your classes by ratio.'
        )

    split_train = len(files) - sum(fixed)
    split_val = split_train + fixed[0]

    li = split_files(files, split_train, split_val, len(fixed) == 2)
    copy_files(li, class_dir, output, prog_bar)
    return len(files)

def split_class_dir_ratio(class_dir, output, fulloutput, ratio, seed, prog_bar):
    """Splits one very class folder
    """
    files = setup_files(class_dir, seed)

    split_train = int(ratio[0] * len(files))
    split_val = split_train + int(ratio[1] * len(files))

    li = split_files(files, split_train, split_val, len(ratio) == 3)
    copy_files(li, class_dir, output, fulloutput, prog_bar)


def split_files(files, split_train, split_val, use_test):
    """Splits the files along the provided indices
    """
    files_train = files[:split_train]
    files_val = files[split_train:split_val] if use_test else files[split_train:]

    li = [(files_train, "train"), (files_val, "test")]

    # optional test folder
    if use_test:
        files_test = files[split_val:]
        li.append((files_test, "test"))
    return li


def copy_files(files_type, class_dir, output, fulloutput, prog_bar):
    """Copies the files from the input folder to the output folder
    """
    # get the last part within the file
    for (files, folder_type) in files_type:
        full_path = os.path.join(output, folder_type, fulloutput)

        pathlib.Path(full_path).mkdir(parents=True, exist_ok=True)
        for f in files:
            if not prog_bar is None:
                prog_bar.update()
            shutil.copy2(f, full_path)
            
            
ratio('/Drive', output='/Drive/data/', seed=1337, ratio=(.8, .2))  #the partition to 80% train 20% test


## Augmentation Class

In [0]:
class SubPolicy(object):
    def __init__(self, p1, operation1, magnitude_idx1, p2, operation2, magnitude_idx2, fillcolor=(128, 128, 128)):
        ranges = {
            "shearX": np.linspace(0, 0.3, 10),
            "shearY": np.linspace(0, 0.3, 10),
            "translateX": np.linspace(0, 150 / 331, 10),
            "translateY": np.linspace(0, 150 / 331, 10),
            "rotate": np.linspace(0, 30, 10),
            "color": np.linspace(0.0, 0.9, 10),
            "posterize": np.round(np.linspace(8, 4, 10), 0).astype(np.int),
            "solarize": np.linspace(256, 0, 10),
            "contrast": np.linspace(0.0, 0.9, 10),
            "sharpness": np.linspace(0.0, 0.9, 10),
            "brightness": np.linspace(0.0, 0.9, 10),
            "autocontrast": [0] * 10,
            "equalize": [0] * 10,
            "invert": [0] * 10
        }

        # from https://stackoverflow.com/questions/5252170/specify-image-filling-color-when-rotating-in-python-with-pil-and-setting-expand
        def rotate_with_fill(img, magnitude):
            rot = img.convert("RGBA").rotate(magnitude)
            return Image.composite(rot, Image.new("RGBA", rot.size, (128,) * 4), rot).convert(img.mode)

        func = {
            "shearX": lambda img, magnitude: img.transform(
                img.size, Image.AFFINE, (1, magnitude * random.choice([-1, 1]), 0, 0, 1, 0),
                Image.BICUBIC, fillcolor=fillcolor),
            "shearY": lambda img, magnitude: img.transform(
                img.size, Image.AFFINE, (1, 0, 0, magnitude * random.choice([-1, 1]), 1, 0),
                Image.BICUBIC, fillcolor=fillcolor),
            "translateX": lambda img, magnitude: img.transform(
                img.size, Image.AFFINE, (1, 0, magnitude * img.size[0] * random.choice([-1, 1]), 0, 1, 0),
                fillcolor=fillcolor),
            "translateY": lambda img, magnitude: img.transform(
                img.size, Image.AFFINE, (1, 0, 0, 0, 1, magnitude * img.size[1] * random.choice([-1, 1])),
                fillcolor=fillcolor),
            "rotate": lambda img, magnitude: rotate_with_fill(img, magnitude),
            # "rotate": lambda img, magnitude: img.rotate(magnitude * random.choice([-1, 1])),
            "color": lambda img, magnitude: ImageEnhance.Color(img).enhance(1 + magnitude * random.choice([-1, 1])),
            "posterize": lambda img, magnitude: ImageOps.posterize(img, magnitude),
            "solarize": lambda img, magnitude: ImageOps.solarize(img, magnitude),
            "contrast": lambda img, magnitude: ImageEnhance.Contrast(img).enhance(
                1 + magnitude * random.choice([-1, 1])),
            "sharpness": lambda img, magnitude: ImageEnhance.Sharpness(img).enhance(
                1 + magnitude * random.choice([-1, 1])),
            "brightness": lambda img, magnitude: ImageEnhance.Brightness(img).enhance(
                1 + magnitude * random.choice([-1, 1])),
            "autocontrast": lambda img, magnitude: ImageOps.autocontrast(img),
            "equalize": lambda img, magnitude: ImageOps.equalize(img),
            "invert": lambda img, magnitude: ImageOps.invert(img)
        }

        # self.name = "{}_{:.2f}_and_{}_{:.2f}".format(
        #     operation1, ranges[operation1][magnitude_idx1],
        #     operation2, ranges[operation2][magnitude_idx2])
        self.p1 = p1
        self.operation1 = func[operation1]
        self.magnitude1 = ranges[operation1][magnitude_idx1]
        self.p2 = p2
        self.operation2 = func[operation2]
        self.magnitude2 = ranges[operation2][magnitude_idx2]


    def __call__(self, img):
        if random.random() < self.p1: img = self.operation1(img, self.magnitude1)
        if random.random() < self.p2: img = self.operation2(img, self.magnitude2)
        return img


class ImageNetPolicy(object):
    """ Randomly choose one of the best 24 Sub-policies on ImageNet.
        Example:
        >>> policy = ImageNetPolicy()
        >>> transformed = policy(image)
        Example as a PyTorch Transform:
        >>> transform=transforms.Compose([
        >>>     transforms.Resize(256),
        >>>     ImageNetPolicy(),
        >>>     transforms.ToTensor()])
    """
    def __init__(self, fillcolor=(128, 128, 128)):
        self.policies = [
            SubPolicy(0.4, "posterize", 8, 0.6, "rotate", 9, fillcolor),
            SubPolicy(0.6, "solarize", 5, 0.6, "autocontrast", 5, fillcolor),
            SubPolicy(0.8, "equalize", 8, 0.6, "equalize", 3, fillcolor),
            SubPolicy(0.6, "posterize", 7, 0.6, "posterize", 6, fillcolor),
            SubPolicy(0.4, "equalize", 7, 0.2, "solarize", 4, fillcolor),

            SubPolicy(0.4, "equalize", 4, 0.8, "rotate", 8, fillcolor),
            SubPolicy(0.6, "solarize", 3, 0.6, "equalize", 7, fillcolor),
            SubPolicy(0.8, "posterize", 5, 1.0, "equalize", 2, fillcolor),
            SubPolicy(0.2, "rotate", 3, 0.6, "solarize", 8, fillcolor),
            SubPolicy(0.6, "equalize", 8, 0.4, "posterize", 6, fillcolor),

            SubPolicy(0.8, "rotate", 8, 0.4, "color", 0, fillcolor),
            SubPolicy(0.4, "rotate", 9, 0.6, "equalize", 2, fillcolor),
            SubPolicy(0.0, "equalize", 7, 0.8, "equalize", 8, fillcolor),
            SubPolicy(0.6, "invert", 4, 1.0, "equalize", 8, fillcolor),
            SubPolicy(0.6, "color", 4, 1.0, "contrast", 8, fillcolor),

            SubPolicy(0.8, "rotate", 8, 1.0, "color", 2, fillcolor),
            SubPolicy(0.8, "color", 8, 0.8, "solarize", 7, fillcolor),
            SubPolicy(0.4, "sharpness", 7, 0.6, "invert", 8, fillcolor),
            SubPolicy(0.6, "shearX", 5, 1.0, "equalize", 9, fillcolor),
            SubPolicy(0.4, "color", 0, 0.6, "equalize", 3, fillcolor),

            SubPolicy(0.4, "equalize", 7, 0.2, "solarize", 4, fillcolor),
            SubPolicy(0.6, "solarize", 5, 0.6, "autocontrast", 5, fillcolor),
            SubPolicy(0.6, "invert", 4, 1.0, "equalize", 8, fillcolor),
            SubPolicy(0.6, "color", 4, 1.0, "contrast", 8, fillcolor),
            SubPolicy(0.8, "equalize", 8, 0.6, "equalize", 3, fillcolor)
        ]


    def __call__(self, img):
        policy_idx = random.randint(0, len(self.policies) - 1)
        return self.policies[policy_idx](img)

    def __repr__(self):
        return "AutoAugment ImageNet Policy"
      
      

## Create DataSet Class - Class

In [0]:
# DataSet object
import pandas as pd
import os
from torch.utils.data.dataset import Dataset
import imageio
import numpy as np  
from PIL import Image

class ClassDataset(Dataset):
      
  def __init__(self, basedir, transform, percentage):
        super().__init__()
        files = glob.glob(os.path.join(basedir ,'**/*.jpg'), recursive=True)
        if(percentage < 100):
          files_num = len(list(files))
          choosefilesnumber = int(files_num * percentage / 100)
          print(f'choosefilesnumber: {choosefilesnumber}  filesnum: {files_num}')
          print((list(files)))
          selectedfiles = np.random.choice(files, choosefilesnumber, replace=False)
        elif(percentage == 100):
          selectedfiles = files
        
        self.data = pd.DataFrame([self._split_file(f) for f in selectedfiles], 
                            columns=['class_id', 'file_path'])
        names  = np.unique(self.data['class_id'])
        
        self.classDict = {str:index for index, str in enumerate(names)}          
        self.data['class_num'] = self.data['class_id'].map(self.classDict)
        self.transform = transform
        
  def _split_file(self, f):
        parts = f.split(os.sep)[-3:-1]
        return parts[0], f   #label is originaly a str

  def __getitem__(self, index):
      dat = self.data.iloc[index]
      img = Image.open(dat['file_path'])
      if self.transform:
          img = self.transform(img)
      img = np.resize(img, (3,128,128))
      img = np.asarray(img)
      return (img.astype(np.float32), dat['class_num'])
     
  def __len__(self):
      return len(self.data)    

## Create DataSet Class - Instance

In [0]:
# DataSet object
import pandas as pd
import os
from torch.utils.data.dataset import Dataset
import imageio
import numpy as np  
from PIL import Image


class InstanceDataset(Dataset):
    def __init__(self, basedir, transform, percentage):
        super().__init__()
        files = glob.glob(os.path.join(basedir ,'**/*.jpg'), recursive=True)
        if(percentage < 100):
          files_num = len(list(files))
          choosefilesnumber = int(files_num * percentage / 100)
          print(f'choosefilesnumber: {choosefilesnumber}  filesnum: {files_num}')
          print((list(files)))
          selectedfiles = np.random.choice(files, choosefilesnumber, replace=False)
        elif(percentage == 100):
          selectedfiles = files
        
        self.data = pd.DataFrame([self._split_file(f) for f in selectedfiles], 
                            columns=['instance_id', 'file_path'])
        names  = np.unique(self.data['instance_id'])
        
        self.instanceDict = {str:index for index, str in enumerate(names)}          
        self.data['instance_num'] = self.data['instance_id'].map(self.instanceDict)
        self.transform = transform
        
    def _split_file(self, f):
        parts = f.split(os.sep)[-3:-1]
        return parts[1], f   #label is originaly a str

    def __getitem__(self, index):
      dat = self.data.iloc[index]
      #img = imageio.imread(dat['file_path'])
      img = Image.open(dat['file_path'])
      if self.transform:
          img = self.transform(img)
      img = np.resize(img, (3,128,128))
      img = np.asarray(img)
      return (img.astype(np.float32), dat['instance_num'])
     
    def __len__(self):
        return len(self.data)
          

## Create train and test datasets including augmentation

In [0]:
import glob
import torchvision.transforms as transforms
from PIL import Image, ImageEnhance, ImageOps

#basedir = '/content/gdrive/My Drive/video_data/'       
trainbasedir = '/Drive/data/train'   
testbasedir = '/Drive/data/test'

data_transforms = transforms.Compose([transforms.Resize((256,256)),transforms.RandomResizedCrop(224), 
                         transforms.RandomHorizontalFlip(), #ImageNetPolicy(), 
                         transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

#train_set = ClassDataset(trainbasedir, data_transforms, 50)
#test_set = ClassDataset(testbasedir, None, 100)
train_set = InstanceDataset(trainbasedir, data_transforms, 100) 
test_set = InstanceDataset(testbasedir, None, 100)                                                                            

In [0]:
print(len((train_set.data['class_num'])))
print(len(np.unique(train_set.data['class_num'])))
print(max(train_set.data['class_num']))
print(len((test_set.data['class_num'])))
print(len(np.unique(test_set.data['class_num'])))
print(max(test_set.data['class_num']))

40331
23
22
18350
23
22


In [0]:
print(len((train_set.data['instance_num'])))
print(len(np.unique(train_set.data['instance_num'])))
print(max(train_set.data['instance_num']))
print(len((test_set.data['instance_num'])))
print(len(np.unique(test_set.data['instance_num'])))
print(max(test_set.data['instance_num']))

80662
6476
6475
18350
6287
6286


## Create DataLoaders

In [0]:

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable


batch_size = 50

train_loader = torch.utils.data.DataLoader(
                 dataset=train_set,
                 batch_size=batch_size,
                 shuffle=True,
                 num_workers=4)
test_loader = torch.utils.data.DataLoader(
                 dataset=test_set,
                 batch_size=batch_size,
                 shuffle=False,
                 num_workers=4)

print('Train size: {}'.format(len(train_loader)))


Train size: 1614


## Confusion Matrix

In [0]:
def perf_measure(y_actual, y_pred):
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    cm = np.zeros((2,2))
    for i in range(len(y_pred)):
        if y_actual[i]==y_pred[i]:
           TP += 1
        if y_pred[i]==1 and y_actual[i]!=y_pred[i]:
           FP += 1
        if y_actual[i]==y_pred[i]:
           TN += 1
        if y_pred[i]==0 and y_actual[i]!=y_pred[i]:
           FN += 1

        cm[0][0] = TP
        cm[0][1] = FP
        cm[1][0]  = FN
        cm[1][1]  = TN

    return (cm)
  
def create_confusion_matrix_fig(c_cm):
    fig = plt.figure(figsize=(14, 12))
    plt.imshow(c_cm, interpolation='nearest')
    plt.title('Confusion matrix')
    plt.colorbar()
    return fig

## Training

In [0]:
import torch
from torchvision import models
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
import pylab as pl

import time 
import tqdm

use_cuda = torch.cuda.is_available()
assert use_cuda

model = models.resnet18(pretrained=False) #for third net to load our trained instance net

# Writer will output to ./runs/ directory by default
writer = SummaryWriter('with_cm')    
    
num_final_in = model.fc.in_features

NUM_CLASSES_Instance = 6476  #len(np.unique(dataset.data['instance_num']))
NUM_CLASSES_Class = 23

model.fc = nn.Linear(num_final_in, NUM_CLASSES_Instance)      
optimizer = optim.Adam(model.parameters(), lr=0.003) #, momentum=0.9)

criterion = nn.CrossEntropyLoss()

if use_cuda:
  model = model.cuda()
  model.to(torch.device("cuda"))

#model = torch.load('/content/gdrive/My Drive/myModels/InstanceModel.pth')
#num_final_in = model.fc.in_features
#print(f'last fc number: {num_final_in})
#model.fc = nn.Linear(num_final_in, NUM_CLASSES_Instance) 

for epoch in range(50):
    # trainning
    sum_loss = 0 
    total_cnt = 0
    correct_cnt = 0
    tf = time.time()
    for batch_idx, (x, target) in enumerate(train_loader):
    #for batch_idx, (x, target) in enumerate(tqdm.tqdm(train_loader)):
        optimizer.zero_grad()
        if use_cuda:
            x, target = x.cuda(), target.cuda()
        
        out = model(x)
        loss = criterion(out, target)       
        loss.backward()
        optimizer.step()
                
        pred_label = torch.max(out.data, 1)
        total_cnt += x.data.size()[0]
        correct_cnt += (pred_label[1] == target.data).sum()
        sum_loss += loss.data
        
        if (batch_idx+1) % 100 == 0 or (batch_idx+1) == len(train_loader):
            print ('==>>> epoch: {}, batch index: {}, train loss: {:.6f}, acc: {}'.format(
                epoch, batch_idx+1, sum_loss/batch_idx, float(correct_cnt)/total_cnt))    
            
    cm = perf_measure(target.data, pred_label[1])
    cur_fig = create_confusion_matrix_fig(cm)
    writer.add_figure('train_confusion_matrix', cur_fig, global_step=None, close=True, walltime=None)
    writer.add_scalar('train accuracy', float(correct_cnt)/total_cnt, epoch)
    writer.add_scalar('train loss', sum_loss/len(train_loader), epoch)
    
     # testing    
    correct_cnt, sum_loss = 0, 0
    total_cnt = 0
    for batch_idx, (x, target) in enumerate(test_loader):
        x = x.float()
        if use_cuda:
            x, target = x.cuda(), target.cuda()
        out = model(x)   
        loss = criterion(out, target)
        
        _, pred_label = torch.max(out.data, 1)
        total_cnt += x.data.size()[0]
        correct_cnt += (pred_label == target.data).sum()
        # smooth averages
        sum_loss += loss.data
        
        if(batch_idx+1) % 100 == 0 or (batch_idx+1) == len(test_loader):
            print ('==>>> epoch: {}, batch index: {}, test loss: {:.6f}, acc: {:.3f}'.format(
                epoch, batch_idx+1, sum_loss/batch_idx,float(correct_cnt)/total_cnt))
    
    try:
      cm = perf_measure(target.data, pred_label)
      cur_fig = create_confusion_matrix_fig(cm)
      writer.add_figure('test_confusion_matrix', cur_fig, global_step=None, close=True, walltime=None)
    except:
      print("An exception in test_confusion_matrix")
      continue
    writer.add_scalar('test accuracy', float(correct_cnt)/total_cnt, epoch)
    writer.add_scalar('test loss', sum_loss/len(test_loader), epoch)

#torch.save(model.state_dict(), 'yt_bb_detection_train/mymodel3')
writer.close()

elapsed = time.time() - tf
print(f'Elapsed time: {elapsed}')

torch.save(model, '/content/gdrive/My Drive/myModels/newInstanceModel.pth')

# model = models.resnet18(pretrained=True)
# model.load_state_dict(torch.load(PATH))
# model.eval()



      